In [1]:
################
## EXERCICE 1 ##
################

customers = sc.textFile("/FileStore/tables/Customer.txt")
orders = sc.textFile("/FileStore/tables/Order.txt")

print(customers.take(1))

['1,20/09/1985,PARKER LONG']

In [2]:
# REQ_1 : SELECT name FROM Customer WHERE month(startDate)=7
req1 = customers.filter(lambda x: int(x.split(',')[1].split('/')[1]) == 7)
req1 = req1.map(lambda x: x.split(',')[2]).collect()
for i in range(len(req1)):
  print(req1[i])

SAWYER NEWTON
SAWYER NEWTON
DAVIDSON WADE
FULLER OWEN
ROBERTSON BENNETT
SHERMAN KELLY
BUSH PEARSON
BAILEY GARRETT
JACKSON SCOTT
KNIGHT STAFFORD
PECK COLE
CAMPBELL THOMAS
OSBORNE SULLIVAN
RHODES HARVEY
GRIFFIN BOYD
WOOD DAVIS

In [3]:
# REQ_2 : SELECT DISTINCT name FROM Customer WHERE month(startDate)=7
req2 = customers.filter(lambda x: int(x.split(',')[1].split('/')[1]) == 7)
req2 = req2.map(lambda x: x.split(',')[2]).distinct().collect()
for i in range(len(req2)):
  print(req2[i])

SAWYER NEWTON
DAVIDSON WADE
FULLER OWEN
ROBERTSON BENNETT
SHERMAN KELLY
BAILEY GARRETT
KNIGHT STAFFORD
RHODES HARVEY
WOOD DAVIS
BUSH PEARSON
JACKSON SCOTT
PECK COLE
CAMPBELL THOMAS
OSBORNE SULLIVAN
GRIFFIN BOYD

In [4]:
print(orders.take(2))
orders = orders.map(lambda x: (x.split(',')[0],x.split(',')[1]))
print(orders.take(2))

['2615094,1348', '3503263,1238']
[('2615094', '1348'), ('3503263', '1238')]

In [5]:
# REQ_3 : SELECT O.cid, SUM(total), COUNT(DISTINCT total) FROM Order O GROUP BY O.cid
sum_by_id = orders.reduceByKey(lambda x,y: x+y)
total_by_id = orders.groupByKey().map(lambda x: (x[0],len(set(x[1])))) #set(x) suppress doublons from list x
req3 = sum_by_id.join(total_by_id)
print(req3.take(1))
req3 = req3.map(lambda x: (x[0], x[1][0], x[1][1]))
print(req3.take(5))

[('3503263', ('1238', 1))]
[('3503263', '1238', 1), ('513846', '1189', 1), ('1991928', '1314', 1), ('2583510', '1440', 1), ('349164', '1443', 1)]

In [6]:
# REQ_4 : SELECT C.cid, O.total FROM Customer C, Order O WHERE month(startDate)=7 and C.cid=O.cid
req4_custom = customers.filter(lambda x: int(x.split(',')[1].split('/')[1]) == 7).map(lambda x: (x.split(',')[0], (x.split(',')[1],x.split(',')[2])))
print(req4_custom.take(2))
req4 = req4_custom.join(orders)
print(req4.take(2))
req4 = req4.map(lambda x: (x[0],x[1][1]))
print(req4.take(2))

[('2', ('03/07/1994', 'SAWYER NEWTON')), ('200', ('03/07/1994', 'SAWYER NEWTON'))]
[('119', (('13/07/1991', 'CAMPBELL THOMAS'), '1482')), ('119', (('13/07/1991', 'CAMPBELL THOMAS'), '1217'))]
[('119', '1482'), ('119', '1217')]

In [7]:
################
## EXERCICE 2 ##
################

# Given the file urlaverage.txt, design and implement the Spark code (Python) to get the average consultation time of each url page in the file.

urls = sc.textFile("/FileStore/tables/urlaverage.txt")
print(urls.take(5))
urls = urls.map(lambda x: (x.split('\t')[0],x.split('\t')[1]))
#avg_time_by_url = urls.groupByKey().map(lambda x: (x[0], sum(list(x[1]))))
avg_time_by_url = urls.groupByKey().map(lambda x: (x[0], sum([int(x) for x in list(x[1])])/len(list(x[1]))))
print(avg_time_by_url.take(2))

['ccc.it\t18', 'fff.edu\t2', 'bbb.com\t12', 'fff.edu\t3', 'aaa.com\t6']
[('ddd.fr', 16.216216216216218), ('ggg.eu', 15.630434782608695)]

In [8]:
################
## EXERCICE 3 ##
################

#  implement the Word Count algorithm

def only_alphanum(x):
  str=""
  for c in x:
    if c.isalnum() or c==' ':
      str+=c
  return str

doc = sc.textFile("/FileStore/tables/wordcount.txt")
req = doc.map(lambda x : only_alphanum(x))
req = req.filter(lambda x: len(x)>0).map(lambda x: x.strip()) # supress empty values then supress spaces at the begning and end of each string
req = req.flatMap(lambda x: x.split(' '))
print("total nb of words in the doc = {}".format(req.count()))
req = req.map(lambda x: (x,1))
req = req.groupByKey().map(lambda x: (x[0], sum([int(c) for c in x[1]])))
print(req.take(10))

total nb of words in the doc = 124773
[('The', 285), ('Project', 83), ('EBook', 3), ('of', 3694), ('Pride', 6), ('Jane', 263), ('Austen', 4), ('is', 859), ('use', 23), ('anyone', 29)]